In [1]:
import pandas as pd
import ta
import os
import numpy as np
import logging

# Konfigurer logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 🔧 Sti til dine inputfiler
input_folder = r"C:\Users\Kasper Hassing\Desktop\Speciale_KryptoSentiment\data\market_data"
output_folder = input_folder  # Gemmer i samme mappe for nu

# 🔁 For hver coin i listen
coins = ['BTCUSDT']
for coin in coins:
    file_path = os.path.join(input_folder, f'{coin}_ohlcv.csv')
    logging.info(f"Indlæser data fra {file_path}")
    
    # Tjek om filen eksisterer
    if not os.path.exists(file_path):
        logging.error(f"Filen {file_path} eksisterer ikke!")
        continue
        
    # Indlæs data - antag at den har headers
    try:
        df = pd.read_csv(file_path)
        logging.info(f"Indlæste {len(df)} rækker data med headers")
    except:
        # Hvis fejl, prøv at indlæse uden headers og tilføj dem manuelt
        try:
            df = pd.read_csv(file_path, header=None, 
                             names=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            logging.info(f"Indlæste {len(df)} rækker data uden headers")
        except Exception as e:
            logging.error(f"Kunne ikke indlæse fil: {str(e)}")
            continue
    
    # Konverter timestamp til datetime
    try:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    except:
        # Prøv specifikt YYYY-MM-DD format hvis automatisk konvertering fejler
        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d')
        except Exception as e:
            logging.error(f"Kunne ikke konvertere timestamp: {str(e)}")
            continue
    
    # Tilføj dato kolonne
    df['date'] = df['timestamp'].dt.date
    
    logging.info(f"Data dækker perioden: {df['date'].min()} til {df['date'].max()}")
    logging.info(f"Antal rækker: {len(df)}")
    
    # For at undgå NaN-værdier i starten af datasættet for window-baserede indikatorer,
    # vil vi tilføje ekstra historiske data før beregningerne, hvis det er tilgængeligt
    # Her bruger vi en simpel løsning: Gentag de første værdier tilbage i tiden
    
    # 1. Momentum Indikatorer
    logging.info("Beregner momentum indikatorer...")
    rsi_indicator = ta.momentum.RSIIndicator(close=df['close'], window=14)
    df['rsi'] = rsi_indicator.rsi()
    
    # 2. Trend Indikatorer
    logging.info("Beregner trend indikatorer...")
    ema_short = ta.trend.EMAIndicator(close=df['close'], window=12)
    ema_long = ta.trend.EMAIndicator(close=df['close'], window=26)
    df['ema_short'] = ema_short.ema_indicator()
    df['ema_long'] = ema_long.ema_indicator()
    
    # 3. Volatilitet Indikatorer
    logging.info("Beregner volatilitetsindikatorer...")
    atr = ta.volatility.AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14)
    df['volatility_atr'] = atr.average_true_range()
    
    bollinger = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_width'] = (bollinger.bollinger_hband() - bollinger.bollinger_lband()) / bollinger.bollinger_mavg()
    
    # 4. Volumen Indikatorer
    logging.info("Beregner volumen indikatorer...")
    obv = ta.volume.OnBalanceVolumeIndicator(close=df['close'], volume=df['volume'])
    df['obv'] = obv.on_balance_volume()
    
    # For volume_norm, brug .fillna() på resultatet for at undgå NaN i starten
    df['volume_norm'] = df['volume'] / df['volume'].rolling(window=20).mean()
    
    # 5. MACD
    logging.info("Beregner MACD...")
    macd_indicator = ta.trend.MACD(close=df['close'], window_fast=12, window_slow=26, window_sign=9)
    df['macd'] = macd_indicator.macd()
    df['macd_signal'] = macd_indicator.macd_signal()
    df['macd_hist'] = macd_indicator.macd_diff()
    
    # 6. Prisændringer
    logging.info("Beregner prisændringer...")
    df['return_1d'] = df['close'].pct_change(1)
    df['return_3d'] = df['close'].pct_change(3)
    df['return_7d'] = df['close'].pct_change(7)
    
    # 7. Fremtidige returns
    logging.info("Beregner fremtidige returns...")
    # De sidste N dage vil have NaN-værdier for fremtidige returns
    df['future_return_1d'] = df['close'].pct_change(1).shift(-1)
    df['future_return_3d'] = df['close'].pct_change(3).shift(-3)
    df['future_return_7d'] = df['close'].pct_change(7).shift(-7)
    
    # Håndter NaN-værdier
    # 1. For tekniske indikatorer i starten: Fyld med første gyldige værdi
    technical_cols = ['rsi', 'ema_short', 'ema_long', 'volatility_atr', 'bb_width', 
                      'obv', 'volume_norm', 'macd', 'macd_signal', 'macd_hist']
    
    for col in technical_cols:
        first_valid = df[col].first_valid_index()
        if first_valid is not None and first_valid > 0:
            first_value = df.loc[first_valid, col]
            df.loc[:first_valid, col] = first_value
    
    # 2. For returns i starten: Fyld med 0
    return_cols = ['return_1d', 'return_3d', 'return_7d']
    for col in return_cols:
        df[col] = df[col].fillna(0)
    
    # 3. For fremtidige returns i slutningen: Gentag sidste kendte værdi
    future_cols = ['future_return_1d', 'future_return_3d', 'future_return_7d']
    for col in future_cols:
        df[col] = df[col].fillna(method='ffill')
    
    # Tjek om der stadig er NaN-værdier
    nan_count = df.isna().sum().sum()
    if nan_count > 0:
        logging.warning(f"Der er stadig {nan_count} NaN-værdier i datasættet")
        nan_columns = df.isna().sum()
        logging.info(f"NaN-værdier per kolonne:\n{nan_columns[nan_columns > 0]}")
    else:
        logging.info("✅ Ingen NaN-værdier i det endelige datasæt")
    
    # 💾 Gem som ny fil
    output_path = os.path.join(output_folder, f'{coin}_features.csv')
    df.to_csv(output_path, index=False)
    logging.info(f"✅ Gemte fil med indikatorer: {output_path}")
    logging.info(f"Filen indeholder {len(df)} rækker, fra {df['date'].min()} til {df['date'].max()}")

2025-05-09 01:49:22,892 - INFO - Indlæser data fra C:\Users\Kasper Hassing\Desktop\Speciale_KryptoSentiment\data\market_data\BTCUSDT_ohlcv.csv
2025-05-09 01:49:22,896 - INFO - Indlæste 486 rækker data med headers
2025-05-09 01:49:22,900 - INFO - Data dækker perioden: 2020-12-01 til 2022-03-31
2025-05-09 01:49:22,901 - INFO - Antal rækker: 486
2025-05-09 01:49:22,901 - INFO - Beregner momentum indikatorer...
2025-05-09 01:49:22,903 - INFO - Beregner trend indikatorer...
2025-05-09 01:49:22,905 - INFO - Beregner volatilitetsindikatorer...
2025-05-09 01:49:22,911 - INFO - Beregner volumen indikatorer...
2025-05-09 01:49:22,912 - INFO - Beregner MACD...
2025-05-09 01:49:22,914 - INFO - Beregner prisændringer...
2025-05-09 01:49:22,917 - INFO - Beregner fremtidige returns...
C:\Users\Kasper Hassing\AppData\Local\Temp\ipykernel_18928\2066881228.py:127: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[col